In [1]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_KEY = os.getenv("OPENAI_KEY")

# 데이터베이스 로드 함수
def load_vectorstore(persist_directory):
    embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_KEY)
    return Chroma(persist_directory=persist_directory, embedding_function=embeddings)

# 검색 함수
def search(query, vectorstore, n_results=1):
    """
    쿼리에 대한 유사성 검색을 수행하고 결과를 반환합니다.

    Args:
        query (str): 검색할 쿼리
        vectorstore (Chroma): Chroma 벡터 저장소 객체
        n_results (int): 반환할 결과 수

    Returns:
        list: 검색된 문서들의 리스트. 각 문서는 'content'와 'metadata'를 포함.
    """
    # 유사성 검색 수행
    docs = vectorstore.similarity_search(query, k=n_results)

    # 검색된 문서에서 필요한 정보 추출
    results = []
    for doc in docs:
        results.append({
            "content": doc.page_content,  # 문서의 내용
            "metadata": doc.metadata      # 문서의 메타데이터
        })

    return results  # 검색된 문서들의 리스트 반환

In [2]:
persist_directory = "./chroma_db"

try:
    # 벡터 저장소 로드
    vectorstore = load_vectorstore(persist_directory)
    print("벡터 저장소 로드 완료.")

    # 검색 수행
    query = "검색하고자 하는 내용을 입력하세요"
    results = search(query, vectorstore, n_results=3)

    # 검색 결과 출력
    for idx, result in enumerate(results, start=1):
        print(f"결과 {idx}:")
        print(f"내용: {result['content']}")
        print(f"메타데이터: {result['metadata']}")
        print("-" * 50)

except Exception as e:
    print(f"오류 발생: {str(e)}")

C:\Users\dasolkim7\AppData\Local\Temp\ipykernel_14676\3508660060.py:12: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  return Chroma(persist_directory=persist_directory, embedding_function=embeddings)


벡터 저장소 로드 완료.
결과 1:
내용: - 사용자는 2025년 1월 13일 점심에 '선재 업고 튀어라'라는 드라마를 보면서 엄마가 싸준 마라탕 도시락을 먹을 계획이라고 밝혔습니다. 사용자는 매운 음식을 아주 좋아하며, 특히 떡볶이와 짬뽕이 떠오르는 매운 음식이라고 언급하였습니다. 또한, 주말에도 마라탕을 먹을 계획이라고 말하였습니다.
메타데이터: {'date': '2025-01-13', 'topic': '음식'}
--------------------------------------------------
결과 2:
내용: - 사용자는 2025년 1월 12일에 매우 기분이 좋았고, 그냥 좋은 기분이어서 특별한 이유는 없다고 했다.
- 사용자는 맛있는 음식을 먹는 것을 좋아한다. 그날 딸기가 들어간 크림 모카번을 먹었는데, 커피 대신 빵과 함께 먹었다.
- 사용자는 소금빵도 좋아해하며, 마지막으로 먹은 것은 몇 주 전이었다. 그리고 학교 근처의 브레덴코에서 다음에 또 먹을 계획이 있다.
메타데이터: {'date': '2025-01-12', 'topic': '음식'}
--------------------------------------------------
결과 3:
내용: - 주말에는 교회에 다니는 것이 사용자의 계획 중 하나로, 교회에서 가장 좋아하는 활동은 예배드리기라고 언급하였습니다. 또한, 예배 후에는 친구들과 카페에 가기도 하며, 바스크 치즈케이크와 딸기 라떼를 주로 마신다고 말하였습니다.
메타데이터: {'date': '2025-01-13', 'topic': '주말 계획'}
--------------------------------------------------


In [19]:
from openai import OpenAI
import os
from dotenv import load_dotenv
from datetime import datetime

load_dotenv()
OPENAI_KEY = os.getenv("OPENAI_KEY")

client = OpenAI(api_key=OPENAI_KEY)

# 날짜를 자연스러운 표현으로 변환
def convert_date_to_natural_language(date_str):
    today = datetime.today()
    date_obj = datetime.strptime(date_str, "%Y-%m-%d")
    delta = (today - date_obj).days

    if delta <= 7:
        return "저번 주"
    elif delta <= 30:
        return "지난달"
    else:
        return "예전에"

# Assistant 응답 생성 함수
def generate_gpt_response(query, context=None, metadata=None):
    natural_date = None
    if metadata and metadata.get("date"):
        natural_date = convert_date_to_natural_language(metadata["date"])

    system_message = f"""
    너는 사용자와 친근하게 대화하는 어시스턴트 "Melissa"야.

        대화 방식:
        1. 현재 사용자의 말에 먼저 자연스럽게 반응해
        2. 만약 과거 대화 내용과 연관성이 있다면, 그 내용을 자연스럽게 언급해
        3. 현재 대화 주제나 사용자의 관심사를 고려해서 새로운 질문을 해
        
        주의사항:
        - 반말로 대화해
        - 과거 대화는 있을 때만 언급하고, 없으면 현재 대화에만 집중해
        - 날짜 있으면 "며칠 전에", "저번 주에" 같이 자연스럽게 표현해
        - 답변은 간결하게 하되, 기계적이지 않게 해
        - 항상 흥미로운 새 질문으로 마무리해    
        - 과거 대화 내용이 현재 대화 맥락과 유사하지 않은 것 같다면 과감하게 사용하지 마

    현재 사용자의 말: \"{query}\""""
    if context:
        system_message += f"\n과거 대화 내용: \"{context}\" ({natural_date})"

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_message}
        ]
    )
    return response.choices[0].message.content

def search(query, vectorstore, threshold=0.45):
    results = vectorstore.similarity_search_with_score(query, k=3)
    filtered_results = [
        {"content": res[0].page_content, "metadata": res[0].metadata, "score": res[1]}
        for res in results if res[1] >= threshold
    ]
    return filtered_results

def send_message(thread_id, role, content):
    message = client.beta.threads.messages.create(
        thread_id=thread_id,
        role=role,
        content=content
    )
    return message

# 대화 루프
thread = client.beta.threads.create()

# 처음 Assistant 메시지 전송
send_message(thread.id, "assistant", "오늘 하루 어땠어?")

# 사용자의 첫 입력 받기
user_input = input("User: ")

# 사용자의 첫 메시지를 전송
send_message(thread.id, "user", user_input)

# 대화 루프 시작
while True:
    if user_input.lower() in ['quit', 'exit', '종료']:
        break
    
    # 검색
    search_results = search(user_input, vectorstore)
    
    if not search_results:
        print("검색 결과가 없어. 바로 대답할게!")
        assistant_response = generate_gpt_response(user_input)
    else:
        print("검색된 결과:")
        for i, result in enumerate(search_results, start=1):
            print(f"[결과 {i}]")
            print(f"내용: {result['content']}")
            print(f"메타데이터: {result['metadata']}")
            print(f"유사도 점수: {result['score']:.2f}")
            print()

        context = search_results[0]["content"]
        metadata = search_results[0]["metadata"]
        assistant_response = generate_gpt_response(user_input, context, metadata)
    
    print("Assistant:", assistant_response)

    # 다음 사용자 입력
    user_input = input("User: ")
    send_message(thread.id, "user", user_input)

print("대화를 종료할게. 다음에 또 얘기하자!")

검색된 결과:
[결과 1]
내용: - 주말에는 교회에 다니는 것이 사용자의 계획 중 하나로, 교회에서 가장 좋아하는 활동은 예배드리기라고 언급하였습니다. 또한, 예배 후에는 친구들과 카페에 가기도 하며, 바스크 치즈케이크와 딸기 라떼를 주로 마신다고 말하였습니다.
메타데이터: {'date': '2025-01-13', 'topic': '주말 계획'}
유사도 점수: 0.47

Assistant: 오, 동생이랑 스벅 갔구나! 어떤 메뉴 시켰어? 바스크 치즈케이크랑 딸기 라떼도 괜찮겠다, 저번 주에 너가 좋아한다고 했었잖아! 이번에 새로운 메뉴 도전해볼 생각은 없어?
검색 결과가 없어. 바로 대답할게!
Assistant: 우와, 초코칩 프라푸치노 맛있지! 달콤하고 시원해서 여름에 딱이야. 지난번에는 다른 음료를 먹었었지? 이번에는 어떻게 생각하고 주문했어? 다른 음료도 시도해 볼 생각 있어?
검색된 결과:
[결과 1]
내용: - 사용자는 2025년 1월 13일 점심에 '선재 업고 튀어라'라는 드라마를 보면서 엄마가 싸준 마라탕 도시락을 먹을 계획이라고 밝혔습니다. 사용자는 매운 음식을 아주 좋아하며, 특히 떡볶이와 짬뽕이 떠오르는 매운 음식이라고 언급하였습니다. 또한, 주말에도 마라탕을 먹을 계획이라고 말하였습니다.
메타데이터: {'date': '2025-01-13', 'topic': '음식'}
유사도 점수: 0.51

[결과 2]
내용: - 주말에는 교회에 다니는 것이 사용자의 계획 중 하나로, 교회에서 가장 좋아하는 활동은 예배드리기라고 언급하였습니다. 또한, 예배 후에는 친구들과 카페에 가기도 하며, 바스크 치즈케이크와 딸기 라떼를 주로 마신다고 말하였습니다.
메타데이터: {'date': '2025-01-13', 'topic': '주말 계획'}
유사도 점수: 0.53

[결과 3]
내용: - 2025년 1월 11일, 사용자는 마라탕을 먹었다고 전했다. 토핑으로는 푸주를 선택했다. 또한, 딸기 탕후루를 먹었다고 언급했다.이어서 딸기가 들어간 빵을 사먹었다고 말했다